## Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
from bs4 import BeautifulSoup

In [5]:
import requests

## Loading the page

In [6]:
link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [7]:
raw_page=requests.get(link)

In [8]:
page=raw_page.text

In [9]:
#print(page)

## Website scraping using BeautifulSoup

In [9]:
soup = BeautifulSoup(page,"lxml")

In [10]:
#print(soup.prettify())

In [11]:
body = soup.find("tbody")

In [61]:
#print(body.prettify())

In [12]:
rows=body("tr")

allRowItems=BeautifulSoup("<items></items>","lxml")
for row in rows:
    rowItems = row("td")
    for rowItem in rowItems:
        allRowItems.items.append(rowItem)

print(allRowItems.prettify())

<html>
 <body>
  <items>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M1A
     </b>
     <br/>
     <span style="font-size:80%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M2A
     </b>
     <br/>
     <span style="font-size:80%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M3A
     </b>
     <br/>
     <span style="font-size:80%;">
      <a href="/wiki/North_York" title="North York">
       North York
      </a>
      <br/>
      (
      <a href="/wiki/Parkwoods" title="Parkwoods">
       Parkwoods
      </a>
      )
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M4A
     </b>
     <br/>
     <span style="font-size:80%;">
      <a href="/wiki/North_York" title="North York">
       North Y

In [13]:
tds=allRowItems.items("td")
tds
records=[]
for td in tds:
   
    zip_code =td.p.b.text  
    
    texts=td.span.find_all("a")
    
    if len(texts)==0:
        borough="Not assigned"
        neighborhood = "Not assigned"
    elif len(texts)==1:
        borough=texts[0].text
        neighborhood = texts[0].text
    else:
        borough=texts[0].text
        neighborhood=""
        for i in range(len(texts)-1):
            
            neighborhood_part = texts[i+1].text
            if i<(len(texts)-2):
                
                neighborhood+=neighborhood_part+", "
            else:
                neighborhood+=neighborhood_part
            
            
    
    records.append([zip_code,borough,neighborhood])
    print(zip_code)
    print(borough)
    print(neighborhood)
    print()

M1A
Not assigned
Not assigned

M2A
Not assigned
Not assigned

M3A
North York
Parkwoods

M4A
North York
Victoria Village

M5A
Downtown Toronto
Regent Park, Harbourfront

M6A
North York
Lawrence Manor, Lawrence Heights

M7A
Queen's Park
Queen's Park

M8A
Not assigned
Not assigned

M9A
Etobicoke
Islington Avenue

M1B
Scarborough
Malvern, Rouge

M2B
Not assigned
Not assigned

M3B
North York
Don Mills

M4B
East York
Parkview Hill, Woodbine Gardens

M5B
Downtown Toronto
Garden District, Ryerson

M6B
North York
North York

M7B
Not assigned
Not assigned

M8B
Not assigned
Not assigned

M9B
Etobicoke
West Deane Park, Princess Gardens, Martin Grove, Islington

M1C
Scarborough
Rouge Hill, Port Union, Highland Creek

M2C
Not assigned
Not assigned

M3C
North York
Don Mills, Flemingdon Park

M4C
East York
Woodbine Heights

M5C
Downtown Toronto
St. James Town

M6C
York
Humewood-Cedarvale

M7C
Not assigned
Not assigned

M8C
Not assigned
Not assigned

M9C
Etobicoke
Markland Wood

M1E
Scarborough
Guildwo

In [14]:
len(records)

180

## Creating DF

In [15]:
cols=["PostalCode","Borough","Neighborhood"]

In [102]:
toronto = pd.DataFrame(records,columns=cols)

In [103]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [104]:
(toronto["Borough"]=="Not assigned").value_counts()

False    101
True      79
Name: Borough, dtype: int64

In [105]:
toronto.drop(toronto[toronto["Borough"]=="Not assigned"].index, inplace=True)

In [106]:
toronto.reset_index(inplace=True,drop=True)

In [107]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


## Shape of the data frame

In [108]:
toronto.shape

(101, 3)

## Getting Lat and Lng

In [23]:
API_key=" AszI6TqrHFFSvDIjkQ-B-ujLCPCV_Ok2vnoLWZZvGOStF9Jbwwc6DZYKZSw3ZTBP"

In [112]:
for i in range(len(toronto)):
    url='http://dev.virtualearth.net/REST/v1/Locations?postalCode={}&key=AszI6TqrHFFSvDIjkQ-B-ujLCPCV_Ok2vnoLWZZvGOStF9Jbwwc6DZYKZSw3ZTBP'.format(toronto["PostalCode"][i])
    response=requests.get(url).json()
    g_data = response["resourceSets"][0]["resources"][0]["geocodePoints"][0]["coordinates"]
    toronto["Latitude"][i]=g_data[0]
    toronto["Longitude"][i]=g_data[1]
    i=i+1

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [124]:
#toronto["PostalCode"][0]

In [110]:
toronto["Latitude"]=np.nan

In [111]:
toronto["Longitude"]=np.nan

In [125]:
#toronto.head()

In [126]:
#toronto[toronto["PostalCode"]=="M4B"]

In [120]:
#response=requests.get(url1).json()
#response

In [121]:
#response.keys()

In [122]:
#response["resourceSets"][0]["resources"][0]["geocodePoints"][0]["coordinates"]

In [123]:
toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.751255,-79.329895
1,M4A,North York,Victoria Village,43.729958,-79.314201
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655220,-79.361969
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722801,-79.450691
4,M7A,Queen's Park,Queen's Park,43.664486,-79.393021
5,M9A,Etobicoke,Islington Avenue,43.662743,-79.528427
6,M1B,Scarborough,"Malvern, Rouge",43.810154,-79.194603
7,M3B,North York,Don Mills,43.749134,-79.362007
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707577,-79.310913
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657467,-79.377708
